<a href="https://colab.research.google.com/github/styxx216/ML/blob/main/Disaster_tweets_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bert
!pip install transformers

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset
import torch.nn as nn
import torch.optim as optim
import bert
import pandas as pd
import transformers as ppb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
train_path='/content/drive/MyDrive/train (1).csv'

In [ ]:
data=pd.read_csv(train_path)

In [ ]:
data

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [ ]:
print(len(data.loc[data['target'] == 0]))
print(len(data.loc[data['target'] == 1]))
print(len(data['target']))

4342
3271
7613


In [ ]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [ ]:
class text_dataset(Dataset):
  def __init__(self,x,y,max_len=None):
    self.x = x
    self.y = y
    if max_len:
      self.max_len=max_len
    else:
      self.max_len=0
      tokenized = (self.x).apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
      for i in tokenized:
          if len(i) > self.max_len:
              self.max_len = len(i)
        
  def __getitem__(self,index):      
    tokenized = tokenizer.encode(self.x[index], add_special_tokens=True)
    padded = np.array([tokenized + [0]*(self.max_len-len(tokenized))])
    ids = torch.tensor(padded).to(device) 
    sentiment = self.y[index]         
    list_of_labels = torch.tensor(self.y[index] ,dtype=torch.float).to(device) 

    attention_mask = torch.tensor(np.where(padded != 0, 1, 0)).to(device) 
    return ids[-1],attention_mask[-1], list_of_labels
    
  def __len__(self):
    return len(self.x)

In [ ]:
set=text_dataset(data['text'],data['target'])

In [ ]:
set[2][2]

tensor(1., device='cuda:0')

In [ ]:
class BertForClassification(nn.Module):
  def __init__(self,bert='distilbert-base-uncased',num_classes=1,hidden_size=200):
    super(BertForClassification,self).__init__()
    self.BERT=ppb.DistilBertModel.from_pretrained(bert)
    self.dropout = nn.Dropout(0.2)
    #self.df=nn.Linear(768,hidden_size)
    self.classificator=nn.Linear(768,num_classes)
    self.sigmoid = nn.Sigmoid()
  def forward(self,x,mask):
    out=self.BERT(input_ids=x,attention_mask=mask)
    out=out.last_hidden_state
    out = out[:,0,:]
    out=self.dropout(out)
    #out=self.df(out)
    out=self.classificator(out)
    out=self.sigmoid(out)
    return out

In [ ]:
print(set.max_len)

84


In [ ]:

lr=1e-5
model=BertForClassification()
model=model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
loss_fn = nn.BCELoss()
x=set[40][0]
m=set[40][1]
print(x.reshape([1,len(x)]))
print(m)
a=model(x.reshape([1,len(x)]),m.reshape([1,len(x)]))
print(a)
print(a.size())

tensor([[  101,  4638,  2122,  2041,  1024,  8299,  1024,  1013,  1013,  1056,
          1012,  2522,  1013, 25223,  2475,  3619,  4168,  3501,  3501,  8299,
          1024,  1013,  1013,  1056,  1012,  2522,  1013,  1017,  2102,  3501,
          2620,  2480, 14642, 17465,  8299,  1024,  1013,  1013,  1056,  1012,
          2522,  1013, 21076, 10179,  2595, 12879, 15457,  8299,  1024,  1013,
          1013,  1056,  1012,  2522,  1013,  1048, 18413,  3501,  2278,  2620,
          2581,  2243,  4877,  1001, 24978,  2546,  2860,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0]], device='cuda:0')
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([[0

In [ ]:
batch_size=64
validation_split = 0.2
test_set,train_set=torch.utils.data.random_split(set, [int(len(set)*validation_split),len(set)-int(len(set)*validation_split)])

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)

In [ ]:
model.BERT.requires_grad_=True

In [ ]:
epochs=20
for epoch in range(epochs):
  for  batch_idx, (data,mask, target) in enumerate(train_loader):
    optimizer.zero_grad()
    net_out = model(data,mask)
    target=target.reshape(len(target),1)

    loss = loss_fn(net_out, target)
    
    loss.backward()
    optimizer.step()
    
    if batch_idx%20==0:
      for i in range(len(target)):
        print(target[i],net_out[i])

    # test_out=(model(test)>0.5).float()
    # acc=(test_labels==test_out[:,0]).sum().item()
    # acc=acc/len(test_labels)

    if batch_idx % 30 == 0:
        print('Train Epoch: {} [{}/{} ({:.0f}%)]tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                      100. * batch_idx / len(train_loader), loss.data))
        # print(acc)

tensor([0.], device='cuda:0') tensor([0.4226], device='cuda:0', grad_fn=<SelectBackward>)
tensor([0.], device='cuda:0') tensor([0.4106], device='cuda:0', grad_fn=<SelectBackward>)
tensor([1.], device='cuda:0') tensor([0.5158], device='cuda:0', grad_fn=<SelectBackward>)
tensor([1.], device='cuda:0') tensor([0.5417], device='cuda:0', grad_fn=<SelectBackward>)
tensor([0.], device='cuda:0') tensor([0.4873], device='cuda:0', grad_fn=<SelectBackward>)
tensor([1.], device='cuda:0') tensor([0.5085], device='cuda:0', grad_fn=<SelectBackward>)
tensor([0.], device='cuda:0') tensor([0.5051], device='cuda:0', grad_fn=<SelectBackward>)
tensor([0.], device='cuda:0') tensor([0.5428], device='cuda:0', grad_fn=<SelectBackward>)
tensor([1.], device='cuda:0') tensor([0.5273], device='cuda:0', grad_fn=<SelectBackward>)
tensor([0.], device='cuda:0') tensor([0.5097], device='cuda:0', grad_fn=<SelectBackward>)
tensor([0.], device='cuda:0') tensor([0.4728], device='cuda:0', grad_fn=<SelectBackward>)
tensor([1.

KeyboardInterrupt: ignored

In [ ]:
test_path='/content/drive/MyDrive/test (1).csv'

In [ ]:
test_data=pd.read_csv(test_path)

In [ ]:
class text_dataset_test(Dataset):
  def __init__(self,x,id,max_len=84):
    self.x = x
    self.id = id
    if max_len:
      self.max_len=max_len
    else:
      self.max_len=0
      tokenized = (self.x).apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
      for i in tokenized:
          if len(i) > self.max_len:
              self.max_len = len(i)
        
  def __getitem__(self,index):      
    tokenized = tokenizer.encode(self.x[index], add_special_tokens=True)
    padded = np.array([tokenized + [0]*(self.max_len-len(tokenized))])
    ids = torch.tensor(padded).to(device)       
    list_of_labels = torch.tensor(self.id[index] ,dtype=torch.float).to(device) 

    attention_mask = torch.tensor(np.where(padded != 0, 1, 0)).to(device) 
    return ids[-1],attention_mask[-1], list_of_labels
    
  def __len__(self):
    return len(self.x)

In [ ]:
test_set=text_dataset_test(test_data['text'],test_data['id'])

In [ ]:
df = pd.DataFrame(columns=['id', 'target'])

In [ ]:
df

,id,target


In [ ]:
for i in range(len(test_set)):
  x=test_set[i][0]
  m=test_set[i][1]

  out=model(x.reshape([1,len(x)]),m.reshape([1,len(x)]))
  if out>0.5:
    out=1
  else:
    out=0

  df.loc[i]=[int(test_set[i][2]),out]

In [ ]:
df

,id,target
0,0,0
1,2,1
2,3,1
3,9,1
4,11,1
...,...,...
3258,10861,0
3259,10865,1
3260,10868,1
3261,10874,1


In [ ]:
df.keys()

Index(['id', 'target'], dtype='object')

In [ ]:
path='/content/drive/MyDrive/submission.csv'

In [ ]:
df.to_csv(path)

In [ ]:
ss=pd.read_csv(path)

In [ ]:
ss.keys()

Index(['id', 'target'], dtype='object')